In [6]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from tqdm import tqdm

# === Configurații ===
SOURCE_DIR = 'preprocessed_dataset/train'
TARGET_DIR = 'preprocessed_dataset/train_augmented'
AUGMENTATIONS_PER_IMAGE = 5  # câte imagini noi să genereze per imagine originală
IMAGE_SIZE = (224, 224)  # trebuie să corespundă cu modelul tău

# === Setări de augmentare ===
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# === Creează directoarele destinație dacă nu există ===
os.makedirs(TARGET_DIR, exist_ok=True)
CLASSES = os.listdir(SOURCE_DIR)

for class_name in CLASSES:
    os.makedirs(os.path.join(TARGET_DIR, class_name), exist_ok=True)

# === Procesăm fiecare imagine ===
for class_name in CLASSES:
    class_path = os.path.join(SOURCE_DIR, class_name)
    target_class_path = os.path.join(TARGET_DIR, class_name)
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    print(f'Augmentez clasa "{class_name}" ({len(images)} imagini)...')
    
    for img_name in tqdm(images):
        img_path = os.path.join(class_path, img_name)
        try:
            # Încarcă imaginea
            img = load_img(img_path, target_size=IMAGE_SIZE)
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)

            # SOLUȚIA 1: Folosește save_to_dir (cea mai simplă)
            for i in range(AUGMENTATIONS_PER_IMAGE):
                generator = datagen.flow(
                    x, 
                    batch_size=1,
                    save_to_dir=target_class_path,
                    save_prefix=f'aug_{i}_{img_name.split(".")[0]}',
                    save_format='jpg'  # sau 'png'
                )
                next(generator)  # generează un batch
            
            # Dacă vrei control manual, folosește SOLUȚIA 2:
            """
            # SOLUȚIA 2: Control manual cu iterator
            flow_gen = datagen.flow(x, batch_size=1)
            for i in range(AUGMENTATIONS_PER_IMAGE):
                batch = next(flow_gen)
                aug_img = array_to_img(batch[0])
                base_name = img_name.split('.')[0]
                extension = img_name.split('.')[-1]
                aug_img.save(os.path.join(target_class_path, f'aug_{i}_{base_name}.{extension}'))
            """

        except Exception as e:
            print(f'Eroare la imaginea {img_name}: {e}')

print("Augmentare completă!")

Augmentez clasa "adenocarcinoma" (310 imagini)...


100%|██████████| 310/310 [00:07<00:00, 38.90it/s]


Augmentez clasa "large_cell_carcinoma" (156 imagini)...


100%|██████████| 156/156 [00:03<00:00, 39.55it/s]


Augmentez clasa "normal" (156 imagini)...


100%|██████████| 156/156 [00:03<00:00, 39.87it/s]


Augmentez clasa "squamous_cell_carcinoma" (242 imagini)...


100%|██████████| 242/242 [00:06<00:00, 39.84it/s]

Augmentare completă!
